In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
import warnings
warnings.filterwarnings('ignore')

In [21]:
# reading the excel file
df_start = pd.read_excel('worldcities.xlsx')

#get a data frame with selected columns
FORMAT = ['city', 'lat', 'lng','country','iso2','population']
df_selected = df_start[FORMAT]

#insert in the dataframe the flag for the population
df_selected["flg_pop"]=df_selected.population.apply(lambda x: 1 if not(pd.isna(x)) and x > 200000 else 0)#(0 if pd.isna(x) else 0))

# sorting values by longitude
df_selected.sort_values(by=['lng'], inplace=True, ignore_index=True)

# searching the index of London (GB)
rslt_df = df_selected.loc[(df_selected['city'] == 'London') & 
              (df_selected['iso2']=='GB')] 
index_of_london = rslt_df.index[0]

# splitting the dataframe before and after the London index
df_a = df_selected.iloc[:index_of_london]
df_b = df_selected.iloc[index_of_london:]

# reshaping of dataframe
df = df_b.append(df_a).reset_index(drop=True)

In [46]:
class AroundTheWorld(object):
    
    def __init__(self, dataframe, city_start, country_start, n_min=3, x_size=0.2, y_size=0.1):
        self.dataframe=dataframe
        self.coord_step=None #(lng, lat city_step)
        self.lat_max=None
        self.lat_min=None
        self.lng_min=None
        self.lng_max=None
        self.city_start=city_start
        self.country_start=country_start
        self.city_step=city_start
        self.country_step=country_start
        self.n_min=n_min
        self.x_size=x_size
        self.y_size=y_size
        self.hours=0
        self.n_step=0
        self.dataframe["visited_city"]=0
        self.x_size_default=x_size
        self.y_size_default=y_size
        
    def generate_grid(self):
        self.coord_step = tuple(self.dataframe.loc[(self.dataframe['city'] == self.city_step) & (self.dataframe['iso2']== self.country_step)][['lng','lat']].iloc[0]) #midpoint of shortest side of rectangular
        self.lat_max=self.coord_step[1]+self.y_size/2
        self.lat_min=self.coord_step[1]-self.y_size/2
        self.lng_min=self.coord_step[0]
        self.lng_max=self.coord_step[0]+self.x_size
        #if self.lng_max > 180:
         #   self.lng_max -= 360
        # print(self.lat_max, self.lat_min, self.lng_min, self.lng_max)
        
    def query(self): 
        grid_city=self.dataframe.loc[(self.dataframe["lat"] >= self.lat_min) &
                                     (self.dataframe["lat"] <= self.lat_max) & 
                                     (self.dataframe["lng"] <= self.lng_max) & 
                                     (self.dataframe["lng"] >= self.lng_min) & 
                                     (self.dataframe["city"] != self.city_step) &
                                     (self.dataframe["visited_city"] == 0)]
        #print(grid_city)
        return grid_city
    
    def check_grid_city(self):
        self.generate_grid()
        grid_city=self.query()
        n_row=grid_city.shape[0]
        if n_row < self.n_min:
            self.x_size *= 2
            self.y_size *= 2
            print(self.x_size, self.y_size)
            return self.check_grid_city() 
        else: 
            self.x_size = self.x_size_default
            self.y_size = self.y_size_default
            return grid_city
        #print(n_row)
        
    
    def weight(self, grid_city):
        grid_city["distance"]=grid_city.apply(self.calculate_distance, axis=1, point_city_step=self.coord_step )
        grid_city.sort_values(by=['distance'], inplace=True)
        grid_city=grid_city[:self.n_min]
        grid_city["weight"] = [2**x for x in range(1,self.n_min+1)]
        grid_city["weight"] += grid_city.apply(self.calculate_weigth, axis=1, country_step=self.country_step)
        return grid_city
        
    @staticmethod #decorator
    def calculate_weigth(row, country_step):
        pop_weigth=2 if row['flg_pop'] == 1 else 0
        country_weigth=2 if row['iso2'] != country_step else 0
        return pop_weigth + country_weigth
    
    @staticmethod
    def calculate_distance(row, point_city_step):
        return euclidean_distances([list(point_city_step)], [[row['lng'],row['lat']]])[0][0]
             
    def step(self, grid_city):
        grid_city.sort_values(by=["weight","distance"], ascending=[True, False], inplace=True)
        step=grid_city.iloc[0]
        self.coord_step=tuple(step[["lng", "lat"]])
        self.city_step=step["city"]
        self.country_step=step["iso2"]
        self.hours += step["weight"]
        self.dataframe["visited_city"].iloc[step.name] = 1
        print(self.coord_step, self.city_step, self.country_step, self.hours)
        
    def travel(self):
        while ((self.city_start != self.city_step) or (self.country_start != self.country_step) or (self.n_step == 0)):
            grid_city = self.check_grid_city()
            grid_city_weight= self.weight(grid_city)
            self.step(grid_city_weight)
            self.n_step += 1
        print(self.n_step, self.city_step, self.country_step)
            

In [47]:
around_debug = AroundTheWorld(df, "Beringovskiy", "RU", x_size=2, y_size=9)
#around_debug.coord_step=(179.3067, 63.0655)
around_debug.generate_grid()
print(around_debug.lat_max, around_debug.lat_min, around_debug.lng_min, around_debug.lng_max)
k=around_debug.check_grid_city()
k
#around.check_grid_city()

67.5655 58.5655 -180.6933 -178.6933
4 18
8 36


,city,lat,lng,country,iso2,population,flg_pop,visited_city
12972,Zvëzdnyy,70.9566,-179.5900,Russia,RU,10.0,0,0
12973,Egvekinot,66.3221,-179.1837,Russia,RU,2248.0,0,0
12978,Provideniya,64.4235,-173.2258,Russia,RU,2536.0,0,0


In [27]:
around = AroundTheWorld(df, "London", "GB")
#around.generate_grid()
#k=around.query()
#around.check_grid_city()
#h=around.weight(k)
#around.step(h)
around.travel()

(-0.1182, 51.5172) Holborn GB 2
(-0.097, 51.552) Highbury GB 6
(-0.075, 51.5166) Spitalfields GB 8
(-0.0462, 51.5152) Stepney GB 10
(-0.0266, 51.5414) Hackney GB 12
(-0.0155, 51.5606) Leyton GB 14
(-0.0026, 51.5423) Stratford GB 16
(0.0077, 51.5347) West Ham GB 18
(0.0554, 51.5323) East Ham GB 20
(0.08, 51.54) Barking GB 22
(0.0855, 51.5588) Ilford GB 24
(0.1119, 51.5584) Goodmayes GB 26
(0.1422, 51.5397) Dagenham GB 28
(0.1801, 51.5768) Romford GB 30
(0.2128, 51.5565) Hornchurch GB 32
(0.2512, 51.5557) Upminster GB 34
(0.2956, 51.5207) South Ockendon GB 36
(0.33, 51.475) Grays GB 38
(0.3582, 51.4606) Tilbury GB 40
0.4 0.2
(0.3685, 51.4415) Gravesend GB 42
0.4 0.2
(0.37, 51.51) Orsett GB 44
0.4 0.2
(0.4244, 51.514) Stanford le Hope GB 46
0.4 0.2
(0.4886, 51.5761) Basildon GB 48
(0.504, 51.569) Pitsea GB 50
(0.5207, 51.6114) Wickford GB 52
(0.59, 51.57) Thundersley GB 54
(0.6049, 51.5864) Rayleigh GB 56
(0.6095, 51.5535) Hadleigh GB 58
(0.6535, 51.5425) Leigh-on-Sea GB 60
(0.71, 51.55) 

0.8 0.4
(17.2989, 49.7305) Šternberk CZ 386
0.4 0.2
0.8 0.4
(17.7495, 49.7744) Vítkov CZ 388
0.4 0.2
(17.8759, 49.871) Hradec nad Moravici CZ 390
0.4 0.2
(17.9044, 49.9381) Opava CZ 392
0.4 0.2
0.8 0.4
(18.0048, 49.932) Kravaře CZ 394
0.4 0.2
(18.192, 49.8979) Hlučín CZ 396
0.4 0.2
(18.3576, 49.9041) Bohumín CZ 400
0.4 0.2
(18.3763, 49.8662) Rychvald CZ 402
(18.3894, 49.831) Petřvald CZ 404
0.4 0.2
(18.4302, 49.8452) Orlová CZ 406
0.4 0.2
(18.5428, 49.8542) Karviná CZ 408
0.4 0.2
0.8 0.4
(18.6262, 49.7461) Český Těšín CZ 410
0.4 0.2
(18.6708, 49.6776) Třinec CZ 414
0.4 0.2
0.8 0.4
(18.7646, 49.5767) Jablunkov CZ 416
0.4 0.2
0.8 0.4
(18.8667, 49.6667) Wisła PL 420
0.4 0.2
0.8 0.4
1.6 0.8
(19.0444, 49.8225) Bielsko-Biała PL 422
0.4 0.2
0.8 0.4
1.6 0.8
(19.2233, 49.8821) Kęty PL 424
0.4 0.2
0.8 0.4
1.6 0.8
(19.2333, 50.0333) Oświęcim PL 426
0.4 0.2
0.8 0.4
(19.275, 50.2) Jaworzno PL 428
0.4 0.2
0.8 0.4
(19.4, 50.1333) Chrzanów PL 430
0.4 0.2
0.8 0.4
(19.6333, 50.1333) Krzeszowice PL 432
0

3.2 1.6
(29.4736, 53.2686) Kirawsk BY 726
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(30.0333, 52.9) Zhlobin BY 728
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(30.5667, 52.7167) Buda-Kashalyova BY 730
0.4 0.2
0.8 0.4
1.6 0.8
(31.1808, 52.5592) Vyetka BY 734
0.4 0.2
0.8 0.4
(31.3167, 52.4167) Dobrush BY 736
0.4 0.2
0.8 0.4
1.6 0.8
(31.7333, 52.4333) Zlynka RU 740
0.4 0.2
0.8 0.4
1.6 0.8
(31.9333, 52.5333) Novozybkov RU 742
0.4 0.2
0.8 0.4
1.6 0.8
(32.2361, 52.7528) Klintsy RU 744
0.4 0.2
0.8 0.4
1.6 0.8
(32.4, 53.0167) Surazh RU 746
0.4 0.2
0.8 0.4
1.6 0.8
(32.6833, 52.85) Unecha RU 748
0.4 0.2
0.8 0.4
1.6 0.8
(32.8483, 53.0611) Mglin RU 750
0.4 0.2
0.8 0.4
1.6 0.8
(33.45, 52.9333) Pochëp RU 752
0.4 0.2
0.8 0.4
1.6 0.8
(33.7667, 52.5833) Trubchevsk RU 754
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(34.4939, 52.15) Sevsk RU 756
0.4 0.2
0.8 0.4
1.6 0.8
(35.0756, 52.1289) Dmitriyev-L’govskiy RU 758
0.4 0.2
0.8 0.4
1.6 0.8
(35.3667, 52.3333) Zheleznogorsk RU 760
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(35.8589, 52.0894) Fatezh RU 

3.2 1.6
6.4 3.2
(110.45, 51.35) Khilok RU 1014
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(114.5228, 51.1031) Aginskoye RU 1018
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(115.5666, 50.95) Olovyannaya RU 1020
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(116.3006, 50.5306) Sherlovaya Gora RU 1022
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(116.5333, 50.3833) Borzya RU 1024
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(117.3269, 49.6514) Zabaykalsk RU 1026
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(118.0333, 50.1) Krasnokamensk RU 1030
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(120.172, 50.2411) E’erguna CN 1034
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(121.5167, 50.7833) Genhe CN 1038
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(123.7167, 50.5667) Oroqen Zizhiqi CN 1040
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(125.216, 49.1745) Nenjiang CN 1042
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(125.3, 47.8804) Yi’an CN 1046
0.4 0.2
0.8 0.4
1.6 0.8
(125.866, 48.0263) Keshan CN 1048
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(126.0819, 47.6018) Baiquan CN 1050
0.4 0.

(46.75, 51.7) Marks RU 1324
0.4 0.2
0.8 0.4
1.6 0.8
(47.3833, 52.05) Volsk RU 1326
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(47.7839, 52.039) Balakovo RU 1328
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(48.1, 52.4833) Khvalynsk RU 1330
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(48.4667, 53.1667) Syzran RU 1332
0.4 0.2
0.8 0.4
1.6 0.8
(48.6667, 53.1667) Oktyabrsk RU 1334
0.4 0.2
0.8 0.4
1.6 0.8
(49.5, 53.4) Zhigulevsk RU 1338
0.4 0.2
0.8 0.4
1.6 0.8
(49.9167, 53.1) Novokuybyshevsk RU 1340
0.4 0.2
0.8 0.4
1.6 0.8
(50.6167, 53.2333) Kinel RU 1344
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(51.1667, 52.8) Neftegorsk RU 1346
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(51.35, 53.3667) Otradnyy RU 1348
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(52.2667, 52.7667) Buzuluk RU 1350
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
6.4 3.2
(53.15, 52.4333) Sorochinsk RU 1352
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(55.7833, 52.7667) Kumertau RU 1356
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(55.9328, 52.9647) Meleuz RU 1358
0.4 0.2
0.8 0.4
1.6 0.8
3.2 1.6
(55.9333, 53.3667) Salavat RU 

...

1.6923032801030365e+124 8.461516400515183e+123
3.384606560206073e+124 1.6923032801030365e+124
6.769213120412146e+124 3.384606560206073e+124
1.3538426240824292e+125 6.769213120412146e+124
2.7076852481648584e+125 1.3538426240824292e+125
5.415370496329717e+125 2.7076852481648584e+125
1.0830740992659434e+126 5.415370496329717e+125
2.1661481985318867e+126 1.0830740992659434e+126
4.3322963970637735e+126 2.1661481985318867e+126
8.664592794127547e+126 4.3322963970637735e+126
1.7329185588255094e+127 8.664592794127547e+126
3.465837117651019e+127 1.7329185588255094e+127
6.931674235302038e+127 3.465837117651019e+127
1.3863348470604075e+128 6.931674235302038e+127
2.772669694120815e+128 1.3863348470604075e+128
5.54533938824163e+128 2.772669694120815e+128
1.109067877648326e+129 5.54533938824163e+128
2.218135755296652e+129 1.109067877648326e+129
4.436271510593304e+129 2.218135755296652e+129
8.872543021186608e+129 4.436271510593304e+129
1.7745086042373216e+130 8.872543021186608e+129
3.549017208474643e+

1.0373788922202483e+179 5.1868944611012415e+178
2.0747577844404966e+179 1.0373788922202483e+179
4.149515568880993e+179 2.0747577844404966e+179
8.299031137761986e+179 4.149515568880993e+179
1.6598062275523973e+180 8.299031137761986e+179
3.3196124551047946e+180 1.6598062275523973e+180
6.639224910209589e+180 3.3196124551047946e+180
1.3278449820419178e+181 6.639224910209589e+180
2.6556899640838356e+181 1.3278449820419178e+181
5.311379928167671e+181 2.6556899640838356e+181
1.0622759856335343e+182 5.311379928167671e+181
2.1245519712670685e+182 1.0622759856335343e+182
4.249103942534137e+182 2.1245519712670685e+182
8.498207885068274e+182 4.249103942534137e+182
1.6996415770136548e+183 8.498207885068274e+182
3.3992831540273096e+183 1.6996415770136548e+183
6.798566308054619e+183 3.3992831540273096e+183
1.3597132616109238e+184 6.798566308054619e+183
2.7194265232218477e+184 1.3597132616109238e+184
5.4388530464436954e+184 2.7194265232218477e+184
1.0877706092887391e+185 5.4388530464436954e+184
2.1755

3.974446316289815e+232 1.9872231581449075e+232
7.94889263257963e+232 3.974446316289815e+232
1.589778526515926e+233 7.94889263257963e+232
3.179557053031852e+233 1.589778526515926e+233
6.359114106063704e+233 3.179557053031852e+233
1.2718228212127408e+234 6.359114106063704e+233
2.5436456424254817e+234 1.2718228212127408e+234
5.087291284850963e+234 2.5436456424254817e+234
1.0174582569701927e+235 5.087291284850963e+234
2.0349165139403853e+235 1.0174582569701927e+235
4.0698330278807707e+235 2.0349165139403853e+235
8.139666055761541e+235 4.0698330278807707e+235
1.6279332111523083e+236 8.139666055761541e+235
3.2558664223046165e+236 1.6279332111523083e+236
6.511732844609233e+236 3.2558664223046165e+236
1.3023465689218466e+237 6.511732844609233e+236
2.6046931378436932e+237 1.3023465689218466e+237
5.2093862756873864e+237 2.6046931378436932e+237
1.0418772551374773e+238 5.2093862756873864e+237
2.0837545102749546e+238 1.0418772551374773e+238
4.167509020549909e+238 2.0837545102749546e+238
8.335018041

6.090821257125e+286 3.0454106285625e+286
1.218164251425e+287 6.090821257125e+286
2.43632850285e+287 1.218164251425e+287
4.8726570057e+287 2.43632850285e+287
9.7453140114e+287 4.8726570057e+287
1.94906280228e+288 9.7453140114e+287
3.89812560456e+288 1.94906280228e+288
7.79625120912e+288 3.89812560456e+288
1.559250241824e+289 7.79625120912e+288
3.118500483648e+289 1.559250241824e+289
6.237000967296e+289 3.118500483648e+289
1.2474001934592e+290 6.237000967296e+289
2.4948003869184e+290 1.2474001934592e+290
4.9896007738368e+290 2.4948003869184e+290
9.9792015476736e+290 4.9896007738368e+290
1.99584030953472e+291 9.9792015476736e+290
3.99168061906944e+291 1.99584030953472e+291
7.98336123813888e+291 3.99168061906944e+291
1.596672247627776e+292 7.98336123813888e+291
3.193344495255552e+292 1.596672247627776e+292
6.386688990511104e+292 3.193344495255552e+292
1.2773377981022208e+293 6.386688990511104e+292
2.5546755962044415e+293 1.2773377981022208e+293
5.109351192408883e+293 2.5546755962044415e+29

KeyboardInterrupt: 

In [25]:
df[12970:]

,city,lat,lng,country,iso2,population,flg_pop
12970,Beringovskiy,63.0655,179.3067,Russia,RU,1861.0,0
12971,Labasa,-16.4333,179.3667,Fiji,FJ,27949.0,0
12972,Zvëzdnyy,70.9566,-179.5900,Russia,RU,10.0,0
12973,Egvekinot,66.3221,-179.1837,Russia,RU,2248.0,0
12974,Leava,-14.2933,-178.1583,Wallis And Futuna,WF,NaN,0
...,...,...,...,...,...,...,...
26564,Coulsdon,51.3211,-0.1386,United Kingdom,GB,25695.0,0
26565,Bogandé,12.9690,-0.1380,Burkina Faso,BF,9854.0,0
26566,Benidorm,38.5342,-0.1314,Spain,ES,68721.0,0
26567,Brighton,50.8429,-0.1313,United Kingdom,GB,134293.0,0


In [ ]:
plt.figure(figsize=(200,100))
plt.scatter(df.lng,df.lat, color="green", marker="o", alpha=0.5)
plt.scatter(k.lng,k.lat,color="red", marker="+")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid()
plt.show()